In [1]:
import pandas as pd
path = 'C:\\Users\\liwei\Desktop\\Fort Baker\\FBCM Operations\\02012022\\'
enfusion_filename = 'Daily_position_FBCMF.csv'
risk_mmonitor_filename = 'Risk Monitor.xlsx'

In [2]:
raw_df = pd.read_csv(path + enfusion_filename)
risk_df = pd.read_excel(path + risk_mmonitor_filename, sheet_name = 'RISK', usecols= 'A, B, D, V, W')

In [3]:
# data feed from enfusion
clean_df = raw_df[['FundName', 'FB Book Name', 'FB Issuer Ticker', 'FB Nominal Quantity', 'Instrument Type']].copy()
clean_df.dropna(inplace = True)
clean_df = clean_df[(clean_df['Instrument Type']!='FX Forward') & (clean_df['Instrument Type']!='Cash')]
clean_df.reset_index(drop = True, inplace = True)
clean_df['FB Nominal Quantity'] = clean_df['FB Nominal Quantity'].astype('float')
clean_df['FB Issuer Ticker'] = clean_df['FB Issuer Ticker'].str.upper()


In [4]:
# current data in risk monitor
risk_df.dropna(axis = 0, how = 'all', inplace = True)
risk_df.drop(risk_df.tail(15).index, inplace = True)
risk_df.drop(risk_df.head(9).index, inplace = True)
risk_df.columns = ['FB Issuer Ticker', 'ACQ Ticker', 'FB Book Name', 'TGT', 'ACQ']
risk_df['TGT'] = risk_df['TGT'].astype('float')
risk_df['ACQ'] = risk_df['ACQ'].astype('float')

In [5]:
row_headers = ['RELATIVE VALUE CATALYST', 'CONVERTS', 'MANDATORY', 'PRE DEAL SPACS', 'SPAC WARRANTS', 'WARRANTS']
temp_risk_df=risk_df[risk_df['FB Issuer Ticker'].isin(row_headers)==False]

colnames = ['FB Issuer Ticker', 'FB Book Name', 'FB Nominal Quantity']
ACQ_df = temp_risk_df[['ACQ Ticker', 'FB Book Name', 'ACQ']]
ACQ_df.columns = colnames
TGT_df = temp_risk_df[['FB Issuer Ticker', 'FB Book Name', 'TGT']]
TGT_df.columns = colnames
clean_risk_df = ACQ_df.append(TGT_df).copy()

In [6]:
ACQ_group_df = ACQ_df.groupby(['FB Issuer Ticker', 'FB Book Name']).sum()
TGT_group_df = TGT_df.groupby(['FB Issuer Ticker', 'FB Book Name']).sum()

In [7]:
merge_df = TGT_group_df.merge(ACQ_group_df, on = ['FB Issuer Ticker', 'FB Book Name'], how = 'outer')
merge_df = merge_df.fillna(0)

In [8]:
closed_df = merge_df.loc[(merge_df==0).all(axis=1)]
open_raw_df = merge_df.loc[~(merge_df==0).all(axis=1)]

In [9]:
%%time
open_list = []
for i in range(open_raw_df.shape[0]):
    if open_raw_df.iloc[i, 0] * open_raw_df.iloc[i, 1] == 0:
        open_list.append(open_raw_df.iloc[i, 0]+open_raw_df.iloc[i, 1])
    else: 
        open_list.append(max(open_raw_df.iloc[i, 0], open_raw_df.iloc[i, 1]))

open_group_df = pd.DataFrame(open_list, index = open_raw_df.index, columns = ['FB Nominal Quantity'])

Wall time: 14 ms


In [10]:
clean_group_df = clean_df.groupby(['FB Issuer Ticker', 'FB Book Name']).sum()
clean_open_df = clean_group_df.loc[~(clean_group_df==0).all(axis=1)]

In [ ]:
open_group_df.merge(clean_group_df, how='inner')

In [13]:
diff = pd.concat([open_group_df,clean_group_df]).drop_duplicates(keep=False)

In [14]:
diff

,,FB Nominal Quantity
FB Issuer Ticker,FB Book Name,
SCMA PRIVATE,Pre Deal SPACs,32500.0


In [ ]:
open_group_df.sort_values(['FB Issuer Ticker'], ascending = True).to_csv(path+'open.csv')

In [ ]:
clean_open_df.sort_values(['FB Issuer Ticker'], ascending = True).to_csv(path+'clean.csv')